<a href="https://colab.research.google.com/github/limweiliang/weed-classifier/blob/main/CS3244_ProjectCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tfds-nightly tensorflow-datasets tensorflow matplotlib

In [4]:
!nvidia-smi

Sat Oct 23 12:59:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   60C    P8    10W /  N/A |   1086MiB /  6144MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import sklearn
import math

In [6]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
BATCH_SIZE = 64

In [8]:
train_ds, validation_ds, test_ds = tfds.load('deep_weeds', as_supervised= True, shuffle_files= True, split=["train[:70%]", "train[:20%]", "train[:10%]"], batch_size = BATCH_SIZE)

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

Number of training samples: 192
Number of validation samples: 55
Number of test samples: 28


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

In [10]:
train_ds = train_ds.shuffle(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE).cache()
validation_ds = validation_ds.shuffle(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE).cache()
test_ds = test_ds.shuffle(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE).cache()

In [11]:
from tensorflow.keras import datasets, layers, models, regularizers
import datetime

In [19]:
model = models.Sequential()

#Re-scale inputs to 0-1
model.add(layers.Rescaling(1/255, input_shape = (256,256,3)))

#Data Augmentation
model.add(layers.RandomFlip("horizontal_and_vertical"))
model.add(layers.RandomRotation((-0.2,0.2), fill_mode = "nearest"))

model.add(layers.Conv2D(96, (11, 11), activation='relu', input_shape=(256, 256, 3), strides = 4))
model.add(layers.MaxPooling2D((3, 3), strides = 2))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(256, (5, 5), activation='relu', padding = 'same'))
model.add(layers.MaxPooling2D((3, 3), strides = 2))

model.add(layers.Conv2D(256, (5, 5), activation='relu', padding = 'same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding = 'same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding = 'same'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(192, (3, 3), activation='relu', padding = 'same'))
model.add(layers.Conv2D(192, (3, 3), activation='relu', padding = 'same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding = 'same'))
model.add(layers.MaxPooling2D((3, 3), strides = 2))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.25))

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.25))

model.add(layers.Dense(9))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 256, 256, 3)       0         
_________________________________________________________________
random_flip_1 (RandomFlip)   (None, 256, 256, 3)       0         
_________________________________________________________________
random_rotation_1 (RandomRot (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 62, 62, 96)        34944     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 96)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 30, 30, 96)        384       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 30, 30, 256)      

In [21]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
%load_ext tensorboard

In [34]:
%tensorboard --logdir logs/fit

In [23]:
EarlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True)
model.fit(train_ds, epochs=200, 
                    validation_data=validation_ds, callbacks = [tensorboard_callback, EarlyStop], batch_size = BATCH_SIZE)

Epoch 1/200
192/192 [==============================] - 23s 115ms/step - loss: 1.7236 - accuracy: 0.5175 - val_loss: 2.9041 - val_accuracy: 0.5263
Epoch 2/200
192/192 [==============================] - 23s 120ms/step - loss: 1.3975 - accuracy: 0.5336 - val_loss: 2.1505 - val_accuracy: 0.4415
Epoch 3/200
192/192 [==============================] - 23s 120ms/step - loss: 1.3256 - accuracy: 0.5393 - val_loss: 1.4516 - val_accuracy: 0.4900
Epoch 4/200
192/192 [==============================] - 23s 119ms/step - loss: 1.2738 - accuracy: 0.5540 - val_loss: 1.5362 - val_accuracy: 0.4537
Epoch 5/200
192/192 [==============================] - 23s 119ms/step - loss: 1.2262 - accuracy: 0.5663 - val_loss: 1.4274 - val_accuracy: 0.4906
Epoch 6/200
192/192 [==============================] - 24s 122ms/step - loss: 1.1861 - accuracy: 0.5771 - val_loss: 1.8228 - val_accuracy: 0.4121
Epoch 7/200
192/192 [==============================] - 24s 125ms/step - loss: 1.1524 - accuracy: 0.5867 - val_loss: 1.7255 -

Epoch 57/200
192/192 [==============================] - 26s 133ms/step - loss: 0.4730 - accuracy: 0.8376 - val_loss: 0.6689 - val_accuracy: 0.7744
Epoch 58/200
192/192 [==============================] - 25s 131ms/step - loss: 0.4645 - accuracy: 0.8388 - val_loss: 0.8110 - val_accuracy: 0.7193
Epoch 59/200
192/192 [==============================] - 25s 132ms/step - loss: 0.4341 - accuracy: 0.8467 - val_loss: 1.0778 - val_accuracy: 0.6808
Epoch 60/200
192/192 [==============================] - 26s 133ms/step - loss: 0.4367 - accuracy: 0.8439 - val_loss: 0.8558 - val_accuracy: 0.7396
Epoch 61/200
192/192 [==============================] - 26s 133ms/step - loss: 0.4383 - accuracy: 0.8473 - val_loss: 0.8112 - val_accuracy: 0.7467
Epoch 62/200
192/192 [==============================] - 25s 132ms/step - loss: 0.4297 - accuracy: 0.8515 - val_loss: 0.8011 - val_accuracy: 0.7470
Epoch 63/200
192/192 [==============================] - 26s 132ms/step - loss: 0.4542 - accuracy: 0.8421 - val_loss: 0

192/192 [==============================] - 26s 134ms/step - loss: 0.2266 - accuracy: 0.9230 - val_loss: 0.5989 - val_accuracy: 0.8555
Epoch 113/200
192/192 [==============================] - 26s 137ms/step - loss: 0.2177 - accuracy: 0.9277 - val_loss: 0.6155 - val_accuracy: 0.8338
Epoch 114/200
192/192 [==============================] - 26s 134ms/step - loss: 0.2105 - accuracy: 0.9309 - val_loss: 0.5069 - val_accuracy: 0.8501
Epoch 115/200
192/192 [==============================] - 27s 138ms/step - loss: 0.2074 - accuracy: 0.9297 - val_loss: 0.7176 - val_accuracy: 0.8021
Epoch 116/200
192/192 [==============================] - 26s 136ms/step - loss: 0.5726 - accuracy: 0.8243 - val_loss: 2.1499 - val_accuracy: 0.4643
Epoch 117/200
192/192 [==============================] - 26s 136ms/step - loss: 0.9140 - accuracy: 0.6869 - val_loss: 1.2328 - val_accuracy: 0.5997
Epoch 118/200
192/192 [==============================] - 26s 136ms/step - loss: 0.6915 - accuracy: 0.7651 - val_loss: 0.7622 -

192/192 [==============================] - 27s 142ms/step - loss: 0.1445 - accuracy: 0.9539 - val_loss: 1.1237 - val_accuracy: 0.7564
Epoch 168/200
192/192 [==============================] - 27s 140ms/step - loss: 0.1293 - accuracy: 0.9571 - val_loss: 45.5221 - val_accuracy: 0.8438
Epoch 169/200
192/192 [==============================] - 28s 143ms/step - loss: 0.1562 - accuracy: 0.9489 - val_loss: 0.5959 - val_accuracy: 0.8507
Epoch 170/200
192/192 [==============================] - 27s 142ms/step - loss: 0.1301 - accuracy: 0.9570 - val_loss: 0.6268 - val_accuracy: 0.8441
Epoch 171/200
192/192 [==============================] - 27s 141ms/step - loss: 0.3077 - accuracy: 0.9076 - val_loss: 0.9240 - val_accuracy: 0.7627
Epoch 172/200
192/192 [==============================] - 28s 146ms/step - loss: 0.1401 - accuracy: 0.9534 - val_loss: 0.5472 - val_accuracy: 0.8575
Epoch 173/200
192/192 [==============================] - 27s 138ms/step - loss: 0.1294 - accuracy: 0.9558 - val_loss: 0.6511 

In [29]:
model.save('./CNN_weights/CNN_RRR_Epoch200.h5')

In [24]:
model.evaluate(validation_ds)

55/55 [==============================] - 2s 36ms/step - loss: 0.3182 - accuracy: 0.9086


[0.3182007670402527, 0.9086236357688904]

In [25]:
y_pred = model.predict(test_ds)
y_pred = np.argmax(y_pred, axis=1) # Convert one-hot to index

In [26]:
test = tfds.as_numpy(test_ds)
y_pred = []
y_label = []

for i in test:
  flat_img = i[0]
  flat_label = i[1]
  pred = model.predict(flat_img)
  pred = np.argmax(pred, axis=1)
  y_pred.extend(pred)
  y_label.extend(flat_label)

y_pred = np.array(y_pred)
y_label = np.array(y_label)

In [27]:
len(y_pred)

1751

In [28]:
from sklearn.metrics import classification_report

print(y_pred)
print(classification_report(y_label, y_pred))

[7 1 8 ... 2 8 2]
              precision    recall  f1-score   support

           0       0.80      0.64      0.71        98
           1       0.89      0.97      0.93       108
           2       0.94      0.78      0.85       101
           3       0.96      0.85      0.90       114
           4       0.78      0.99      0.87        91
           5       0.96      0.88      0.92       109
           6       0.96      0.87      0.91       105
           7       0.76      0.90      0.82        98
           8       0.94      0.96      0.95       927

    accuracy                           0.91      1751
   macro avg       0.89      0.87      0.87      1751
weighted avg       0.92      0.91      0.91      1751

